In [ ]:
import sys
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis')

In [ ]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import scipy
import importlib
import pickle

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
import importlib.util

# Set up environment (e.g. sys.path)
assembler_folder = r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler'
top_folder = r'C:\Users\lwang\Documents\GitHub\DataAnalysis'

# Set up the module function to be tested. This is equivalent to from X.Y.Z import module_func, but avoid potential ImportError
background_func_file_path = os.path.join(assembler_folder,'Modules','glucoseModel','channel_gluc_current','time_varying_background.py')
background_func_name = os.path.relpath(background_func_file_path,top_folder).replace('.py','').replace(os.sep,'.')
background_func_spec = importlib.util.spec_from_file_location(background_func_name,background_func_file_path)
background_func_info = importlib.util.module_from_spec(background_func_spec)
background_func_spec.loader.exec_module(background_func_info)
background_func = getattr(background_func_info,'background_gamma_cdf_based')

sensitivity_func_file_path = os.path.join(assembler_folder,'Modules','glucoseModel','channel_dermis_gluc','time_varying_sensitivity.py')
sensitivity_func_name = os.path.relpath(sensitivity_func_file_path,top_folder).replace('.py','').replace(os.sep,'.')
sensitivity_func_spec = importlib.util.spec_from_file_location(sensitivity_func_name,sensitivity_func_file_path)
sensitivity_func_info = importlib.util.module_from_spec(sensitivity_func_spec)
sensitivity_func_spec.loader.exec_module(sensitivity_func_info)
sensitivity_func = getattr(sensitivity_func_info,'sensitivity_gamma_cdf_based')

load data

In [ ]:
df=pd.read_csv(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\agg_mfg_ref_sample_df_from17.csv')
df_orig=df.copy()

Setting

In [ ]:
import scipy.stats as dist
dist_info_background = dist.gamma(a=1,loc=0,scale=1/20)
synthetic_background = 0.05 - 0.04*dist_info_background.cdf(df_orig['time_from_start_featureGeneration'])
dist_info_slope = dist.gamma(a=1,loc=0,scale=1/5)
synthetic_slope = 0.1 + 0.9*dist_info_slope.cdf(df_orig['time_from_start_featureGeneration'])
df_orig['cur1'] = df_orig['cur1'].mul(synthetic_slope).add(synthetic_background)

In [ ]:
training_group = ['Pre-Piv-2-training'] # ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
test_group = ['Pre-Piv-2-training']
all_training_channel = ['1']
training_time_window = [0.083,5]
training_chemistry = '2.6.11'
test_chemistry = '2.6.11'
iso_thresh = 25
params_bound = [
    [0,1e5],
    [1e-6,0.1],[1e-6,0.1],[1e-2,1e2],
    [0,1e3],[0,1e3],[1e-2,1e2],
    [1e-1,1e2],[1e-1,10],[1e-2,1e2],
]
col_for_optim = [
    'cur1','cur3','cur4','Chemistry',
    'time_from_start_featureGeneration','mag1_1007_featureGeneration','mag3_1007_featureGeneration','mag4_1007_featureGeneration',
    'Study','retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose','cgm_adj','ysi','acck'
]

In [ ]:
def glucose_model(cur,features,params):
    background_params = params['background']
    sensitivity_params = params['sensitivity']
    glucose_current,_ = background_func(cur,features,background_params)
    dermis_gluc,sensitivity_info = sensitivity_func(glucose_current,features,sensitivity_params)
    sensitivity = sensitivity_info['sensitivity']
    return dermis_gluc, sensitivity

def params_list_to_dict(params_list,channel_str):
    params = {}
    background_params = {
        'feature_schema': {'mag_1007':f'mag{channel_str}_1007'},
        'sim_slope_background': {'value': params_list[0]},
        'sim_intercept_background_start': {'value': params_list[1]},
        'sim_intercept_background_end': {'value': params_list[2]},
        'sim_intercept_background_a': {'value': 1},
        'sim_intercept_background_loc': {'value': 0},
        'sim_intercept_background_beta': {'value': params_list[3]},
    }
    params['background'] = background_params
    sensitivity_params = {
        'feature_schema': {'mag_1007':f'mag{channel_str}_1007'},
        'sim_intercept_sensitivity_start': {'value': params_list[4]*1e-10},
        'sim_intercept_sensitivity_end': {'value': params_list[5]*1e-10},
        'sim_intercept_sensitivity_a': {'value': 1},
        'sim_intercept_sensitivity_loc': {'value': 0},
        'sim_intercept_sensitivity_beta': {'value': params_list[6]},
        'sim_slope_sensitivity_start': {'value': params_list[7]*1e-3},
        'sim_slope_sensitivity_end': {'value': params_list[8]*1e-3},
        'sim_slope_sensitivity_a': {'value': 1},
        'sim_slope_sensitivity_loc': {'value': 0},
        'sim_slope_sensitivity_beta': {'value': params_list[9]},
    }
    params['sensitivity'] = sensitivity_params
    return params

def mard(params_list,g,cur,features,channel_str):
    params = params_list_to_dict(params_list,channel_str)
    g_hat, _ = glucose_model(cur,features,params)    
    mard = g_hat.sub(g).abs().div(g).mean()
    return mard

def costFunc(params_list,g,cur,features,channel_str):
    params = params_list_to_dict(params_list,channel_str)
    g_hat, sensitivity = glucose_model(cur,features,params)
    error = g_hat.sub(g).div(g).mul(sensitivity).mul(1e3).abs().mean()
    return error

def callback_da(x,f,context=0):
    params_dict = params_list_to_dict(x,1)
    print(f'Cost function value is {f}, and parameters are {params_dict}')

In [ ]:
df = df_orig[col_for_optim].copy()
df.rename(columns={
                'time_from_start_featureGeneration': 'time_from_start',
                'mag1_1007_featureGeneration': 'mag1_1007',
                'mag3_1007_featureGeneration': 'mag3_1007',
                'mag4_1007_featureGeneration': 'mag4_1007',
                'retro_model_output_s1_calculateGlucose': 'retro_model_output_s1',
                'retro_model_output_s3_calculateGlucose': 'retro_model_output_s3',
                'retro_model_output_s4_calculateGlucose': 'retro_model_output_s4',
                }, inplace=True)
all_results= {}
all_optim_params = {}
hypo_idx = df['cgm_adj'].le(100)
for ch_str in all_training_channel:
    print(f'Optmizing channel {ch_str}')
    this_result = []
    df[f'ad_s{ch_str}'] = df[f'retro_model_output_s{ch_str}'].sub(df['cgm_adj']).abs()
    df[f'ard_s{ch_str}'] = df[f'ad_s{ch_str}'].div(df['cgm_adj']).mul(100)
    df[f'iso_s{ch_str}'] = df[f'ard_s{ch_str}']
    df.loc[hypo_idx,f'iso_s{ch_str}'] = df.loc[hypo_idx,f'ad_s{ch_str}']
    col_for_optim_no_ysi_acck = list(df.columns)
    col_for_optim_no_ysi_acck.remove('ysi')
    col_for_optim_no_ysi_acck.remove('acck')
    training_index = df[df['Study'].isin(training_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & 
                            df['Chemistry'].eq(training_chemistry)].index
    df_training = df.loc[training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    valid_training_index = df[df['Study'].isin(training_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & 
                            df[f'iso_s{ch_str}'].le(iso_thresh) & df['Chemistry'].eq(training_chemistry)].index
    df_training_valid = df.loc[valid_training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    print(f'Training data has shape {df_training.shape}')
    print(f'Training data has shape {df_training_valid.shape}')
    good_data_ratio = len(df_training_valid)/len(df_training)
    print(f'Ratio of good data is {good_data_ratio}')
    valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & 
                            df[f'iso_s{ch_str}'].le(iso_thresh) & df['Chemistry'].eq(training_chemistry)].index
    df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    print(f'Test data has shape {df_test_valid.shape}')
    g_train = df_training_valid['cgm_adj']
    features_train = df_training_valid[['time_from_start',f'mag{ch_str}_1007']]
    cur_train = df_training_valid[f'cur{ch_str}']

    g_test = df_test_valid['cgm_adj']
    features_test = df_test_valid[['time_from_start',f'mag{ch_str}_1007']]
    cur_test = df_test_valid[f'cur{ch_str}']

    result = scipy.optimize.dual_annealing(mard,params_bound,callback=callback_da,args=(g_train,cur_train,features_train,ch_str),maxiter=int(1e3))

    this_result.append(mard(result.x,g_train,cur_train,features_train,ch_str))
    this_result.append(mard(result.x,g_test,cur_test,features_test,ch_str))

    params_dict = params_list_to_dict(result.x,ch_str)
    g_hat_train, _ = glucose_model(cur_train,features_train,params_dict) 
    g_hat_test, _ = glucose_model(cur_test,features_test,params_dict) 

    reg_train = LinearRegression().fit(g_train.values.reshape(-1,1), g_hat_train.values)
    this_result.append(reg_train.score(g_train.values.reshape(-1,1), g_hat_train.values))
    this_result.append(reg_train.coef_)
    this_result.append(reg_train.intercept_)           

    reg_test = LinearRegression().fit(g_test.values.reshape(-1,1), g_hat_test.values)
    this_result.append(reg_test.score(g_test.values.reshape(-1,1), g_hat_test.values))
    this_result.append(reg_test.coef_)
    this_result.append(reg_test.intercept_)

    all_optim_params[f'channel_{ch_str}'] = params_dict
    all_results[f'channel_{ch_str}'] = this_result
    
results_df = pd.DataFrame.from_dict(all_results)
results_df.to_csv(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\optimize_time_varying_model_exp_synthetic.csv',header=True)
with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params_exp_synthetic.p','wb') as fp:
    pickle.dump(all_optim_params,fp)    

In [ ]:
with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params.p','wb') as fp:
    pickle.dump(all_optim_params,fp)

In [ ]:
# with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params.p','rb') as fp:
#     all_optim_params2 = pickle.load(fp)

In [ ]:
params_bound

In [ ]:
all_optim_params

In [ ]:
import scipy.stats as dist
dist_info_sim_slope = dist.gamma(a=16,loc=0,scale=1/7061)
time_from_start = [0,0.08,0.25,0.5,1,2,5]
sim_slope_sensitivity = 0 + 1*dist_info_sim_slope.cdf(time_from_start)
print(sim_slope_sensitivity) 

In [ ]:
result

In [ ]:
import pprint
pp = pprint.PrettyPrinter(depth=4)
params_dict = params_list_to_dict(result.x,1)
pp.pprint(params_dict)

In [ ]:
params = params_list_to_dict(result.x,ch_str)
g_train_hat, sensitivity = glucose_model(cur_train,features_train,params)    
mard = g_train_hat.sub(g_train).abs().div(g_train).mean()
error = g_train_hat.sub(g_train).mul(sensitivity).abs().div(g_train).mean()

In [ ]:
mard

In [ ]:
error

In [ ]:
sim_slope = g_train.div(cur_train).div(features_train['mag1_1007']).mean()
sim_slope

In [ ]:
g_simple = cur_train.mul(features_train['mag1_1007']).mul(sim_slope)
g_simple.sub(g_train).abs().div(g_train).mean()

In [ ]:
reg_simple = LinearRegression().fit(g_train.values.reshape(-1,1), g_simple.values)
print(reg_simple.score(g_train.values.reshape(-1,1), g_simple.values))
print(reg_simple.coef_)
print(reg_simple.intercept_)

In [ ]:
params = params_list_to_dict(result.x,ch_str)
g_hat_train, _ = glucose_model(cur_train,features_train,params) 
g_hat_test, _ = glucose_model(cur_test,features_test,params) 
reg_train = LinearRegression().fit(g_train.values.reshape(-1,1), g_hat_train.values)
print(reg_train.score(g_train.values.reshape(-1,1), g_hat_train.values))
print(reg_train.coef_)
print(reg_train.intercept_)